In [ ]:
from cling import cling, bash

**Listing 4.1**

Caption: Compute kernels for the Mandelbrot ans Julia set.

In [ ]:
%%cling
#include <complex>
#include <config.hpp>
//-----------------------------------------------------------------
// Kernel to compute the Mandelbrot set
size_t mandelbrot(std::complex<double> c) { 
  std::complex<double> z(0, 0);
  for (size_t i = 0; i < max_iterations; i++) {
    z = z * z + c;
    if (abs(z) > 2.0) {
      return i;
    }
  }
  return 0;
}

// Kernel to compute the Julia set
size_t julia(std::complex<double> z) { 
  std::complex<double> c(-0.4, 0.6);
  for (size_t i = 0; i < max_iterations; i++) {
    z = z * z + c;
    if (abs(z) > 2.0) {
      return i;
    }
  }
  return 0;
}


**Listing 4.2**

Caption: Example for single threaded computation of the fractal sets.

In [ ]:
%%cling
#include <pbm.hpp>
#include <config.hpp>
#include <numeric>
#include <kernel.hpp>
#include <algorithm>
//-----------------------------------------------------------------
// Definition of utility
PBM pbm = PBM(size_x, size_y); 

std::vector<size_t> index_(size_x);
std::iota(index_.begin(), index_.end(), 0); 
std::for_each(index_.begin(), index_.end(), [](size_t i) { 

  complex c =
      complex(0, 4) * complex(i, 0) / complex(size_x, 0) -
      complex(0, 2); 

  for (size_t j = 0; j < size_y; j++) { 
    // Get the number of iterations
    int value = compute_pixel(c + 4.0 * j / size_y - 2.0); 
    // Convert the value to RGB color space
    std::tuple<size_t, size_t, size_t> color = get_rgb(value); 
    // Set the pixel color
    pbm(i, j) =
        make_color(std::get<0>(color),
                   std::get<1>(color),
                   std::get<2>(color));
  }

  // Save the image
  pbm.save("image_serial_" + type + ".pbm");
});
